#### import

In [1]:
import os
import sys
import time
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By

from bs4 import BeautifulSoup

from datetime import datetime

from IPython.display import display, HTML


#### categories

In [2]:
sasa_category_top = {
    #  (personal care)
    '護膚保養': 5886,

    #   (Beauty cosmetic)
    '潮流彩妝': 5925
}

sasa_category = {
    #  (personal care)
    "潔面產品": 5900,
    "爽膚產品": 5889,
    "面膜": 5892,
    "唇部護理": 5894,
    "眼部護理": 5893,
    "防曬產品": 5895,

    #   (Beauty cosmetic)
    "修容": 5932,
    "胭脂": 5940,
    "高光打亮": 5934,
    "遮瑕": 5939,
    "粉底": 5935,
    "碎粉及粉": 5938,
    "定妝噴霧": 5941,
    "眼線": 5942,
    "眼影": 5943,
    "眉毛產品": 5944,
    "睫毛膏": 5945,
    "假睫毛": 5946,
    "眼部遮瑕": 5948,
    "唇部用品": 5928,
    "指甲產品": 5929,
    "卸妝產品": 5930
}

#### functions

In [3]:
def get_category_product_info(sasa_category_item):

    category, url = sasa_category_item

    # Browser open

    # Create Chrome options
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--incognito")  # Open Chrome in incognito mode
    chrome_options.add_argument("--lang=zh-TW")  # Set the language to Traditional Chinese (zh-TW)
    chrome_options.add_argument("--window-position=0,0")  # Set the initial window position

    success = False
    for i in range(5):
        try:
            # Create the Chrome browser with the options
            browser = webdriver.Chrome(options=chrome_options)

            sasa_url = "https://www.sasa.com.hk/v2/official/SalePageCategory/" + str(url) + "?sortMode=Sales"

            browser.get(sasa_url)
            time.sleep(3)

            print('Open {}: {}'.format(category, sasa_url))
            print("Page title was '{}'".format(browser.title))
            success = True
            break

        except:  # catch all exceptions
            if browser:  # if browser is not None
                browser.quit()  # close the browser
            print('1:  Failed to open {}: {}'.format(category, sasa_url))

        if not success:
            print('Failed to open {}: {}'.format(category, sasa_url))
            

    time.sleep(1)

    # add implicit wait
    browser.implicitly_wait(5)

    # try to locate chat button if exist
    # .easychat-chat-dismiss-button-mobile
    try:
        chat_button = browser.find_element(By.XPATH, '//div[@class="easychat-chat-dismiss-button-mobile"]')
        chat_button.click()
        # print('chat button closed')
    except:
        # print('no chat button found')
        pass

    time.sleep(1)

    # try to locate the "我知道了" button if exist
    # .ns-cookie-privacy-agree-btn

    try:
        cookie_button = browser.find_element(By.XPATH, '//div[@id="root"]')
        cookie_button = browser.find_element(By.XPATH, '//div[@class="container-component"]')
        cookie_button = cookie_button.find_element(By.XPATH, '//a[text()="我知道了"]')
        cookie_button.click()
        print('cookie button closed')
    except:
        # print('no cookie button found')
        pass

    time.sleep(1)

    start_time = time.time()

    num_of_links = 0
    links = []
    inner_window_position = 0
    new_inner_window_position = 1

    while num_of_links != len(links) or inner_window_position < new_inner_window_position:
    # while num_of_links < 1:    
        num_of_links = len(links)
        inner_window_position = browser.execute_script("return window.pageYOffset;")

        # scroll up
        browser.execute_script('window.scrollBy(0,-640)') # use seleneium to run JS to scroll up
        time.sleep(1)  # sleep to wait for the page to load

        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        #scroll down
        new_inner_window_position = browser.execute_script("return window.pageYOffset;")

        time.sleep(2)  # sleep to wait for the page to load

        if inner_window_position == new_inner_window_position:
            for i in range(3):
                browser.execute_script('window.scrollBy(0, 4320)') # use seleneium to run JS to scroll down
                time.sleep(2)  # sleep to wait for the page to load
        #     new_inner_window_position = browser.execute_script("return window.pageYOffset;")
        #     print('Scrolled down:', new_inner_window_position)
        # else:
        #     print('Scolled to the bottom of the page')

        links = browser.find_elements(By.XPATH, '//li[@class="column-grid-container__column"]')

    center = browser.find_element(By.XPATH, '//div[@class="layout-center"]')

    links = center.find_elements(By.XPATH, '//li[@class="column-grid-container__column"]')

    print(category, 'Total number of links:', len(links))

    # search for the class name of the product name and status

    div_count = 0  # dev used to count the number of useful divs in the link

    success = False
    class_product_name = ''
    class_product_status = ''

    for i in range(-5, 0):
        if not success:
            try:
                # print('Index:', i)
                for div in links[-1].find_elements(By.XPATH, './/div'):
                    if len(div.find_elements(By.XPATH, './/div')) == 0 and len(div.text) > 0:
                        if div_count == 0:
                            # print('Div count:', div_count)
                            # print('Div class:', div.get_attribute('class'))
                            # print('Div text:', div.text)
                            class_product_name = div.get_attribute('class')
                            font_color = (div.value_of_css_property('color').split('(')[1].split(')')[0]).split(',')
                            font_rgb = []
                            for i in range(3):
                                font_rgb.append(int(font_color[i].strip()))
                            if font_rgb == [255, 255, 255]:  # white
                                status_flag = 1  # not on sale

                                span_element = div.find_element(By.XPATH, './/span')
                                class_product_status = span_element.get_attribute('class')
                                product_status = span_element.text  # 補貨通知 已售完
                                div_count += 1

                            else:
                                class_product_name = div.get_attribute('class')
                                product_name = div.text
                                # print(i, 'Product Name:', class_product_name, product_name)
                                break
                        elif div_count == 1 and status_flag == 1:
                            class_product_name = div.get_attribute('class')
                            product_name = div.text
                            success = True
                            break

            except:
                print('Error:', i)
                pass

    # Use bs4 to get the product name and status

    # Get the html of the page
    center = browser.find_element(By.XPATH, '//div[@class="layout-center"]')

    soup = BeautifulSoup(center.get_attribute('outerHTML'), 'html.parser')

    browser.quit()

    # '//li[@class="column-grid-container__column"]')
    product_cards = soup.find_all('li', class_='column-grid-container__column')

    # create a dictionary to store the product name and status
    product_names = {}
    product_links = {}
    product_statuses = {}

    for index, product_card in enumerate(product_cards):
        # print('Index:', index)
        product_name = product_card.find('div', class_=class_product_name)
        product_link = product_card.find('a')
        product_status = product_card.find('span', class_=class_product_status)
        if product_name:
            # print('Product Name:', product_name.text)
            product_names[index] = product_name.text
        if product_link:
            # print('Product Link:', product_link.get('href'))
            product_links[index] = product_link.get('href')
        if product_status:
            # print('Product Status:', product_status.text)
            product_statuses[index] = product_status.text

    # Create a DataFrame from the product_names dictionary
    df = pd.DataFrame.from_dict(product_names, orient='index', columns=['Product Name'])

    # rank = index + 1

    df['Rank'] = df.index + 1

    df['Category'] = category

    df['Link'] = pd.Series(product_links)
    df['Link'] = "https://www.sasa.com.hk" + df['Link']

    df['Product ID'] = df['Link'].str.split('/').str[-1]

    # Add the product_status to the DataFrame
    df['Product Status'] = pd.Series(product_statuses)

    # Reorder df: Product ID,Rank,Product Name,Link,Status
    df = df[['Category', 'Product ID', 'Rank', 'Product Name', 'Link', 'Product Status']]

    df['Score'] = ((len(df) + 1 - df['Rank']) / len(df) * 100).round(0).astype(int)

    filename = filename = 'sasa/sasa_{}_{}.csv'.format(url, datetime.now().strftime('%Y%m%d'))
    if filename.split('/')[1] in os.listdir('sasa'):
        os.remove(filename)
        print('Old file removed:', filename)
    df.to_csv(filename, index=False)

    print(category, 'Dataframe saved to:', filename.split('/')[1])

    # display(df.head(3))
    display(df.shape)

    return

#### main fucntion

In [ ]:
print('Main: Start')
start_time = time.time()

for s in sasa_category_top.items():
    # if s[0] == '潔面產品' or s[0] == '高光打亮':
        # print(s)
        get_category_product_info(s)

print("Top categories done")

for s in sasa_category.items():
    # if s[0] == '潔面產品' or s[0] == '高光打亮':
        # print(s)
        get_category_product_info(s)

print("Categories done")

print('Main: End')
end_time = time.time()

print(f"Main: Elapsed time: {(end_time - start_time) / 60:,.0f} minutes")

In [ ]:
# print date
import sys
from datetime import datetime

print(datetime.now().strftime('%Y%m%d'))

filename = 'sasa/sasa_{}_{}.csv'.format('test', datetime.now().strftime('%Y%m%d'))

print(filename)